# PDA Project Report (Movie Recommeder System)
To Recommend 10 Movie Item ID's to Each User in test-PDA2019.csv file (Total 1992 Users)

In this Project.I have Used Model-Based Collaborative Filtering Using Matrix Factorization Method.

A well-known matrix factorization method is Singular value decomposition (SVD).
SVD is a nice way to factorizing a matrix into three other matrices (A = UΣVᵀ).

Its is most Widely used method to identify latent (hidden) factors underlying the interactions between users and items .All matrices have an SVD, which makes it more stable than other methods
 

##### References:
https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/

##### References:
https://www.freecodecamp.org/news/singular-value-decomposition-vs-matrix-factorization-in-recommender-systems-b1e99bc73599/


## Importing Libraries and Read Required Datasets
Importing Required Libraries

Reading Each Dataset File using pandas Library

Merge Both Dataframes rating_data and users_data on userID to know the number of Movies Item ID's i.e already given ratings by the user.

In [38]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from scipy.sparse.linalg import svds

In [39]:
# Reading  User Ratings file and Storing it in rating_data Variable as DataFrame
rating_data = pd.read_csv('/Users/macuser/Desktop/DA Project/data/train-PDA2019.csv')

# Reading No of Users file and Storing it in user_data Variable as DataFrame
users_data = pd.read_csv('/Users/macuser/Desktop/DA Project/data/test-PDA2019.csv')

# Reading movies Details file and Storing it in movies_data Variable as DataFrame
movies_data = pd.read_csv('/Users/macuser/Desktop/DA Project/data/content-PDA2019.csv')
df=pd.merge(users_data,rating_data, on='userID')
df.head()

,userID,recommended_itemIDs,itemID,rating,timeStamp
0,1,260 912 593 1196 858 908 1617 1097 1193 ...,919,4,978301467
1,1,260 912 593 1196 858 908 1617 1097 1193 ...,1721,4,978300150
2,1,260 912 593 1196 858 908 1617 1097 1193 ...,608,4,978301445
3,1,260 912 593 1196 858 908 1617 1097 1193 ...,2797,4,978302039
4,1,260 912 593 1196 858 908 1617 1097 1193 ...,1545,4,978824176


## Checking the Number of Users and Item IDs
From the Merge Dataframe "df" now calculating the Number of Unique Users and item IDs to Know How many Users and Items We have Exactly.

In [40]:
#Checking Total Unique Number of Users 
n_users = df.userID.unique().shape[0]
#Checking Total Unique Number of Item IDs
n_items = df.itemID.unique().shape[0]
#Print Number of Users and Item IDs
print ('Number of users=' + str(n_users) + ' | Number of Item IDs = ' + str(n_items))

Number of users=1992 | Number of Item IDs = 1715


## Creating Pivot Table
Creating the Pivot table from the Merged Dataframe "df" and Store in Ratings Variable

we convert our df table into a 2D matrix and fill the missing values with zeros.

In [41]:
Ratings_pt = df.pivot(index = 'userID', columns ='itemID', values = 'rating').fillna(0)
Ratings_pt.head()

itemID,89,93,94,95,97,98,100,101,102,104,...,3929,3930,3931,3932,3937,3938,3945,3946,3950,3952
userID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Creating Matrix and Computing Sparsity Level of Dataframe df

First Creating Matrix of Above Pivot Table and taking de-mean the data (normalize by each users mean) and convert it from a dataframe to a numpy array.

Also Calculating the Sparsity Level of df Dataframe

In [42]:
R = Ratings_pt.as_matrix()
user_rat_mean = np.mean(R, axis = 1)
Rat_demeaned = R - user_rat_mean.reshape(-1, 1)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [43]:
sparsity = round(1.0 - len(df) / float(n_users * n_items), 3)
print ('The sparsity level of dataset is ' +  str(sparsity * 100) + '%')

The sparsity level of dataset is 99.0%


## Making Predictions Using Singular Value Decomposition (SVds)

I’m going to use the Scipy function svds because it let’s me choose how many latent factors I want to use to approximate the Original df ratings matrix

Computing the Predicted Ratings For Each Item ID Using Rank K In this Case i have decided to go with the Value 3 as i get Most Relevant Predict Ratings. 

Reference=https://en.wikipedia.org/wiki/Low-rank_approximation

The Sigma returned the values instead of a diagonal matrix. This is useful, but since i need to leverage matrix multiplication to get predictions I'll convert it to the diagonal matrix form using diag function.

Now I need to make movie ratings predictions for every user. I can do it all at once by following the math and matrix multiply $U$, $Sigma$, and $V^T$ back to get the rank $k=3$ approximation of $R$.

In [44]:
U, sigma, Vt = svds(Rat_demeaned, k = 3)

In [45]:
sigma = np.diag(sigma)

In [46]:
users_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_rat_mean.reshape(-1, 1)


## Making Item Recommendations Based on Prediction Matrix

Now Creating predictions matrix for every user, I can build a function to recommend item IDs for each user. All I need to do is return the movies with the highest predicted rating that the specified user hasn't already rated.

I'll also return the list of movies the user has already rated, for the sake of comparison.


In [47]:
preds = pd.DataFrame(users_predicted_ratings,columns = Ratings_pt.columns)
preds.head()

itemID,89,93,94,95,97,98,100,101,102,104,...,3929,3930,3931,3932,3937,3938,3945,3946,3950,3952
0,-0.009141,-0.010665,0.011485,0.008096,-0.005351,-0.011380,-0.008410,0.002875,-0.011267,0.020911,...,0.018476,-0.004351,-0.009458,0.007552,-0.008081,-0.011229,-0.011811,-0.008535,-0.009547,0.000412
1,-0.010592,-0.014847,0.025780,0.029248,-0.005140,-0.018887,-0.007757,0.011149,-0.018436,0.048527,...,0.027036,0.005134,-0.016682,0.010154,-0.008839,-0.018712,-0.018360,-0.011803,-0.014081,0.002223
2,0.017638,0.018034,0.016366,0.020014,0.017929,0.017252,0.018551,0.016273,0.017369,0.017234,...,0.015116,0.020403,0.016995,0.015977,0.018371,0.017304,0.017690,0.017459,0.016750,0.015481
3,0.031382,0.007931,0.052580,0.036328,0.010183,0.005829,0.019186,0.065667,0.004732,0.075440,...,-0.013702,0.015223,0.001355,-0.009429,0.009532,0.003520,0.005252,0.016155,0.032055,0.049405
4,0.006922,0.003940,0.007906,0.006927,0.003815,0.003496,0.005446,0.010325,0.003369,0.010547,...,-0.002020,0.005094,0.002670,-0.000373,0.004053,0.003183,0.003568,0.004747,0.006805,0.008076


In [48]:
#Defining Function and Passing Required Arguments(Prediction(Passing Prediction matrix),userInd(Passing User Index),userID(Passing user ID),movies_data(Passing the movie Dataframe),df_ratings(Passing Merged df Rating Dataframe),n_recommendations(passing the number of Recomendations needed))
def recommend_items(predictions, userInd,userID, movies_data, df_ratings, n_recommendations):
    
    # Get and sort the user's predictions
    sorted_user_predictions = preds.iloc[userInd].sort_values(ascending=False)
    
    # Get the user's ID data and Matching it with df Dataframe UserIDs and Store it in User_data.So it Will get Only Those UserIDs Which are Required to be Recommended
    user_data = df_ratings[df_ratings.userID == (userID)]
    
    # Printing With Recommending 10 Items to Each User
    print ('Recommending {0} predicted Item IDs to User {1}'.format(n_recommendations,userID))
    
    # Recommend the highest 10 Item IDs that the user hasn't seen yet.
   
    recommendations = (movies_data[~movies_data['itemID'].isin(user_data['itemID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index()).
         rename(columns = {userInd: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:n_recommendations, :-1]
                      )
    
    
    return recommendations

## Passing User Data and Store in Dataframe Column

Passing Each UserID and User Index to Function to Calculate 10 Predicted Item IDs to Recommend.

Storing the Recommendation as a List in Numpy Array For Each User.

Storing the Recommendation in Required Column Next to Each user in users DataFrame.


In [26]:
liste=[]
i=0
for ind in users_data.index: 
    
    userInd=ind
    userID=users_data['userID'][ind]
    predictions = recommend_items(preds,userInd,userID, movies_data, df, 10)
    liste.append(predictions['itemID'])
    lis=np.array(liste)
    
    users_data['recommended_itemIDs'][i]=lis[i]
    i=i+1

Recommending 10 predicted Item IDs to User 1
Recommending 10 predicted Item IDs to User 3
Recommending 10 predicted Item IDs to User 11


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellAp

Recommending 10 predicted Item IDs to User 29
Recommending 10 predicted Item IDs to User 31
Recommending 10 predicted Item IDs to User 33


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellAp

Recommending 10 predicted Item IDs to User 35
Recommending 10 predicted Item IDs to User 51
Recommending 10 predicted Item IDs to User 53


...

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Recommending 10 predicted Item IDs to User 12063


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Recommending 10 predicted Item IDs to User 12073


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


## Cleaning the Data
Cleaning the Data to Come it into Required Shape as Needed By Removing Unnecessary Symbols

In [36]:
users_data["recommended_itemIDs"]=users_data["recommended_itemIDs"].astype(str)
users_data["recommended_itemIDs"]=users_data["recommended_itemIDs"].str.replace(',',' ')
users_data["recommended_itemIDs"]=users_data["recommended_itemIDs"].str.replace('[',' ')
users_data["recommended_itemIDs"]=users_data["recommended_itemIDs"].str.replace(']',' ')
users_data.head()

,userID,recommended_itemIDs
0,1,260 912 593 1196 858 908 1617 1097 1193 ...
1,3,260 858 912 1198 919 593 589 1197 1617 ...
2,11,589 260 1198 1196 1291 1240 2916 2628 592 ...
3,29,2858 2762 2028 593 1580 1198 457 2959 1617 ...
4,31,2858 2762 2028 318 1198 1580 457 2959 593 ...


## Saving the Data
Storing the Recommended Item IDs in the Required File From users Dataframe

In [37]:
users_data.to_csv('/Users/macuser/Desktop/DA Project/data/test-PDA2019.csv',index=False)